# Welcome to my notebook!

Default code from Kaggle Notebook:

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
   
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   

## Displaying some important libraries:

In [2]:
import tensorflow as tf
print("Tensorflow:", tf.__version__)

import kerastuner as kt
print("kerastuner:", kt.__version__)

import keras_tuner as kt2
print("keras_tuner:", kt2.__version__)

import platform
print("Python:", platform.python_version())

import numpy as np
print("numpy:", np.__version__)

import pandas as pd
print("pandas:", pd.__version__)

import sklearn
print("sklearn version:", sklearn.__version__)

import sklearn
print("sklearn path:", sklearn.__path__)

import matplotlib
print("matplotlib:", matplotlib.__version__)

import seaborn as sns
print("seaborn:", sns.__version__)

# On WSL

# 2024-01-30 11:17:52.768682: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
# 2024-01-30 11:17:53.149956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
# 2024-01-30 11:17:53.150001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
# 2024-01-30 11:17:53.210606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
# 2024-01-30 11:17:53.339576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
# To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
# 2024-01-30 11:17:54.568146: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
# Tensorflow: 2.15.0
# /tmp/ipykernel_3814/2917868046.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
#   import kerastuner as kt
# kerastuner: 1.0.5
# keras_tuner: 1.3.5
# Python: 3.10.12
# numpy: 1.24.3
# pandas: 2.1.4
# sklearn version: 1.2.2
# sklearn path: ['/home/michaelye22/.local/lib/python3.10/site-packages/sklearn']
# matplotlib: 3.8.2
# seaborn: 0.13.0



2024-02-01 21:25:16.555008: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 21:25:16.901198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 21:25:16.901266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 21:25:16.962174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 21:25:17.084277: I tensorflow/core/platform/cpu_feature_guar

Tensorflow: 2.15.0


/tmp/ipykernel_541/2944158885.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


kerastuner: 1.0.5
keras_tuner: 1.3.5
Python: 3.10.12
numpy: 1.24.3
pandas: 2.1.4
sklearn version: 1.2.2
sklearn path: ['/home/michaelye22/.local/lib/python3.10/site-packages/sklearn']
matplotlib: 3.8.2
seaborn: 0.13.0


## Set Global random seed to make sure we can replicate any model that we create (no randomness)

In [3]:
import random
import tensorflow as tf
import numpy as np
import os



np.random.seed(42)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

# **Data Preprocessing:**

### Loading the training and testing data

train_values are the features (X), and train_labels is the target/label (Y)

In [4]:
train_X = pd.read_csv("train_features.csv")
train_y = pd.read_csv("train_labels.csv")

test_values = pd.read_csv("test_features.csv")

# print("train labels:\n", train_Y.head())

# print("train values:\n", train_X.head())
      
# print("test_values:\n", test_values.head())

### Check to see if there are any missing values in the data. If so, we have to do imputation

In [5]:
missing_train_X = train_X.isnull().sum().sum()
print("Number of missing values in train_X:", missing_train_X)

missing_train_y = train_y.isnull().sum().sum()
print("Number of missing values in train_Y:", missing_train_y)

missing_test_values = test_values.isnull().sum().sum()
print("Number of missing values in test_values:", missing_test_values )

Number of missing values in train_X: 0
Number of missing values in train_Y: 0
Number of missing values in test_values: 0


Since we have 0 missing values in each dataframe, we don't have to do imputation!

## Stratified train_test split

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the datasets

train_features = train_X
train_labels = train_y

# Merge the datasets on 'id'
merged_data = pd.merge(train_features, train_labels, on='id')

# Identifying the animal present in each image and creating a combined category
animal_columns = ['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']
merged_data['animal'] = merged_data[animal_columns].idxmax(axis=1)
merged_data['site_animal'] = merged_data['site'] + '_' + merged_data['animal']

# Checking the number of instances for each site_animal combination
combination_counts = merged_data['site_animal'].value_counts()
rare_combinations = combination_counts[combination_counts < 5]

# Separating the dataset into common and rare combinations
common_combinations = merged_data[~merged_data['site_animal'].isin(rare_combinations.index)]
rare_combinations_data = merged_data[merged_data['site_animal'].isin(rare_combinations.index)]

# Stratified split for common combinations
common_train_set, common_test_set = train_test_split(
    common_combinations, test_size=0.25, stratify=common_combinations['site_animal'], random_state=42)

# Randomly splitting rare combinations
total_samples = rare_combinations_data.shape[0]
train_samples = int(np.round(total_samples * 0.75))
rare_train_set = rare_combinations_data.sample(n=train_samples, random_state=42)
rare_test_set = rare_combinations_data.drop(rare_train_set.index)

# Combining the splits into final train and test sets
final_train_set = pd.concat([common_train_set, rare_train_set])
final_test_set = pd.concat([common_test_set, rare_test_set])

# Optional: Verifying the final distribution (can be commented out for large datasets)
final_train_distribution = final_train_set['site_animal'].value_counts(normalize=True)
final_test_distribution = final_test_set['site_animal'].value_counts(normalize=True)
final_distribution_summary = pd.DataFrame({
    'Train Distribution': final_train_distribution,
    'Test Distribution': final_test_distribution
})
print(final_distribution_summary.head())


                        Train Distribution  Test Distribution
site_animal                                                  
S0001_bird                        0.000647           0.000728
S0001_blank                       0.000404           0.000243
S0001_leopard                     0.003073           0.003153
S0001_monkey_prosimian            0.001051           0.000970
S0002_bird                        0.000566           0.000485


## Verify the 75/25 train_test split

In [7]:
# Calculate the number of samples in each set
num_train_samples = final_train_set.shape[0]
num_test_samples = final_test_set.shape[0]
total_samples = num_train_samples + num_test_samples

# Calculate the proportions
train_proportion = num_train_samples / total_samples
test_proportion = num_test_samples / total_samples

# Print out the proportions
print("Training Set Proportion: {:.2%}".format(train_proportion))
print("Test Set Proportion: {:.2%}".format(test_proportion))


Training Set Proportion: 74.99%
Test Set Proportion: 25.01%


## Making sure that train and test set have a 75/25 split for each site

In other words, for each site, around 75% should be in the training data and around 25% should be in the test data. This ensures niether the train or test set have a unbalanced amount of a certain site, leading to bias and bad predictions

In [8]:
# Calculate the count of each site in both sets
site_counts_train = final_train_set['site'].value_counts()
site_counts_test = final_test_set['site'].value_counts()

# Combine the counts into a single DataFrame for comparison
combined_site_counts = pd.DataFrame({'Train Count': site_counts_train, 'Test Count': site_counts_test})

# Calculate the total counts for each site
combined_site_counts['Total Count'] = combined_site_counts['Train Count'] + combined_site_counts['Test Count']

# Calculate the percentage split for each site
combined_site_counts['Train Percentage'] = (combined_site_counts['Train Count'] / combined_site_counts['Total Count']) * 100
combined_site_counts['Test Percentage'] = (combined_site_counts['Test Count'] / combined_site_counts['Total Count']) * 100

# Display the combined counts with percentage split
#print(combined_site_counts.head())
combined_site_counts.to_csv("site_percentage_check.csv")

# Check to see if there any rows of data with a train percentage below 70 or above 80 (the ideal is 75)
filtered_df = combined_site_counts[(combined_site_counts['Train Percentage'] < 70) | (combined_site_counts['Train Percentage'] > 80)]
#print(filtered_df)
#print(len(filtered_df)) # There are only 13 sites which have a bad train/test split, but they all side more towards the train set, which is good


#print(len(final_train_set))


There are only 13 sites which have a bad train/test split, but they all side more towards the train set, which is good

## Making sure the trian and test set have the 75/25 split for each animal 

In other words, for each animal, around 75% should be in the training data and around 25% should be in the test data. This ensures neither the train or test set have a unbalanced amount of a certain animal, leading to bias and bad predictions

In [9]:
# List of label columns
label_columns = ['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']

# Calculate the count of each label in both sets
label_counts_train = final_train_set[label_columns].sum()
label_counts_test = final_test_set[label_columns].sum()

# Combine the counts into a single DataFrame for comparison
combined_label_counts = pd.DataFrame({'Train Count': label_counts_train, 'Test Count': label_counts_test})

# Calculate the total counts for each label
combined_label_counts['Total Count'] = combined_label_counts['Train Count'] + combined_label_counts['Test Count']

# Calculate the percentage split for each label
combined_label_counts['Train Percentage'] = (combined_label_counts['Train Count'] / combined_label_counts['Total Count']) * 100
combined_label_counts['Test Percentage'] = (combined_label_counts['Test Count'] / combined_label_counts['Total Count']) * 100

# Display the combined counts with percentage split
#print(combined_label_counts)


Since each class has around a 75/25 split for train and test split, my data looks good!

## Modifying the data to include only the original columns

In [10]:
#print(final_train_set)

# Remove all the new features that I created for the stratified train_test split
train_X = final_train_set[['id', 'filepath', 'site']]
train_Y = final_train_set[['id','antelope_duiker','bird','blank','civet_genet','hog','leopard','monkey_prosimian','rodent']]

test_X = final_test_set[['id', 'filepath', 'site']]
test_Y = final_test_set[['id','antelope_duiker','bird','blank','civet_genet','hog','leopard','monkey_prosimian','rodent']]


# Make "id" the index column
train_X.set_index('id', inplace=True) # inplace = True means that it edits the original dataframe, and no new dataframe is created
train_Y.set_index('id', inplace=True)

test_X.set_index('id', inplace=True)
test_Y.set_index('id', inplace=True)

#print(test_Y)


#print(train_X)
#print(train_Y)


## Make it so that if there already exists a dataset for train_X, train_Y, and test_X, then we will use those (so that each of my models are trained on the same data, making them deterministic)

In [11]:
import os

if not os.path.exists('train_split_X.csv'):
    print("create new directory")
    train_X.to_csv('train_split_X.csv')
    train_Y.to_csv('train_split_Y.csv')
    test_X.to_csv('test_split_X.csv')
    test_Y.to_csv('test_split_Y.csv')

else:
    print("used old directory")
    train_X = pd.read_csv('train_split_X.csv')
    train_Y = pd.read_csv('train_split_Y.csv')
    test_X = pd.read_csv('test_split_X.csv')
    test_Y = pd.read_csv("test_split_Y.csv")

used old directory


## Check to see if each image has the same dimensions since that's important for data preprocessing

In [12]:
# from PIL import Image
# import os

# def check_image_dimensions(directory):
#     image_sizes = {}
#     for img_name in os.listdir(directory):
#         if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')): # now also checks for .tif
#             continue
#         img_path = os.path.join(directory, img_name)
#         with Image.open(img_path) as img:
#             # Get image size
#             size = img.size
#             if size in image_sizes:
#                 image_sizes[size] += 1
#             else:
#                 image_sizes[size] = 1

#     for size, count in image_sizes.items():
#         print(f"For the {directory} directory, {count} images are of dimension {size}")

# # Use it on the train and test data only if this code segment was never ran in this coding session:

# # Use it on the train and test data:
# check_image_dimensions('train_features')
# check_image_dimensions('test_features')


# # # For the train_features directory, different dimensions found: {(160, 120), (960, 515), (640, 335), (960, 540), (640, 360), (360, 215), (160, 95), (360, 240)}
# # # For the test_features directory, different dimensions found: {(960, 515), (160, 120), (640, 335), (960, 540), (320, 215), (640, 360), (360, 240), (320, 240)}


## Resizing all the images 

In [13]:
# I will actually make the resizing function when I use the ImageDataGenertor. 
# By calling the resize function in the ImageDataGenerator, I won't have to save my images in my local folder and waste space. 

## Before I use ImageDataGenerator, I have to format my dataframes so that they are able to be read properly by ImageDataGenertor

This is a multi-class problem so data format has to be modified so that it can be handled by ImageDataGenerator. 

In [14]:
import pandas as pd

# Merge the two datasets
merged_train = pd.merge(train_X, train_Y, on='id')

# Set "id" as the index column
merged_train.set_index("id", inplace = True) # inplace = True means that it edits the original dataframe, and no new dataframe is created

# Convert multi-label columns into a single column, so that this column tells us what animal type the row is 
merged_train['labels'] = merged_train.apply(lambda row: ' '.join([col for col in merged_train.columns[2:] if row[col]==1]), axis=1)


# Do the same thing for the test set (from train-test split)



# Merge the two datasets
merged_test = pd.merge(test_X, test_Y, on='id')

# Set "id" as the index column
merged_test.set_index("id", inplace = True) # inplace = True means that it edits the original dataframe, and no new dataframe is created

# Convert multi-label columns into a single column, so that this column tells us what animal type the row is 
merged_test['labels'] = merged_test.apply(lambda row: ' '.join([col for col in merged_test.columns[2:] if row[col]==1]), axis=1)

print(merged_test)



                             filepath   site  antelope_duiker  bird  blank  \
id                                                                           
ZJ000687  train_features/ZJ000687.jpg  S0043              0.0   0.0    0.0   
ZJ000926  train_features/ZJ000926.jpg  S0138              1.0   0.0    0.0   
ZJ014494  train_features/ZJ014494.jpg  S0122              0.0   0.0    1.0   
ZJ007702  train_features/ZJ007702.jpg  S0141              1.0   0.0    0.0   
ZJ006294  train_features/ZJ006294.jpg  S0150              0.0   0.0    0.0   
...                               ...    ...              ...   ...    ...   
ZJ015363  train_features/ZJ015363.jpg  S0176              0.0   0.0    0.0   
ZJ015565  train_features/ZJ015565.jpg  S0124              0.0   1.0    0.0   
ZJ015926  train_features/ZJ015926.jpg  S0197              1.0   0.0    0.0   
ZJ016248  train_features/ZJ016248.jpg  S0092              0.0   0.0    1.0   
ZJ016482  train_features/ZJ016482.jpg  S0146              0.0   

## Use Keras ImageDataGenerator() on Train/Test split
The ImageDataGenerator not only helps you load images from the disk but also allows you to perform **data augmentation**, which is a technique to increase the diversity of your training set by applying random transformations (like rotation, zoom, flips, etc.) to the images. This is very useful to prevent overfitting and helps the model generalize better.

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np



# Creating an instance of the ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the image pixel values to [0,1]
    #preprocessing_function = custom_resize,  # call the crop function on each image

    # rotation_range=20,  # Up to 20 degrees of rotation
    # brightness_range=[0.8, 1.2],  # Adjust brightness by 20%
    # horizontal_flip=True,  # Horizontal flip
    # fill_mode='nearest' # The value "nearest" for fill_mode means that any empty space will be filled with the nearest pixel values. In other words, it copies the value of the nearest border pixel to fill the gap. 


)

test_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the image pixel values to [0,1]
    #preprocessing_function = custom_resize,  # call the crop function on each image


    # You not supposed to do data augmetation on test data, to replicate real life data. The only data augmentation should be resizing
)

# Flow from dataframe method to load images using the dataframe

train_split_generator = train_datagen.flow_from_dataframe(
    dataframe=merged_train,
    x_col='filepath',
    y_col='labels',
    target_size=(224, 224),  # automatically resizes so no need for custom resize. When I did custom resize, it made my model worse
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=42
)

test_split_generator = test_datagen.flow_from_dataframe(
    dataframe=merged_test, 
    x_col='filepath',
    y_col='labels',
    target_size=(224, 224), # automatically resizes so no need for custom resize. When I did custom resize, it made my model worse
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=42
)

Found 12365 validated image filenames belonging to 8 classes.
Found 4123 validated image filenames belonging to 8 classes.


## Use Keras ImageDataGenerator() on my test data (from test_values.csv)

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np



test_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the image pixel values to [0,1]
    #preprocessing_function = custom_resize,  # call the crop function on each image


    # You not supposed to do data augmetation on test data, to replicate real life data. The only data augmentation should be resizing
)

# Flow from dataframe method to load images using the dataframe


test_original_generator = test_datagen.flow_from_dataframe(
    dataframe=test_values, 
    x_col='filepath',
    #y_col='', # The actual test data has no labels
    target_size=(224, 224),   # automatically resizes so no need for custom resize. When I did custom resize, it made my model worse
    color_mode='rgb',
    class_mode= None, # The actual test data has no labels
    batch_size=32,
    shuffle=False,
    seed=42
)

Found 4464 validated image filenames.


In [ ]:
# import matplotlib.pyplot as plt

# # Retrieve a batch of images
# images = next(train_split_generator)

# # Display the first two images
# plt.figure(figsize=(10, 5))

# # Image 1
# plt.subplot(1, 2, 1)
# plt.imshow(images[0])
# plt.title("Image 1")
# plt.axis('off')

# # Image 2
# plt.subplot(1, 2, 2)
# plt.imshow(images[1])
# plt.title("Image 2")
# plt.axis('off')

# plt.show()


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from PIL import Image
# import numpy as np
# import matplotlib.pyplot as plt

# class ImageDataGeneratorWithPaths(ImageDataGenerator):
#     def flow_from_dataframe(self, *args, **kwargs):
#         base_generator = super().flow_from_dataframe(*args, **kwargs)
#         while True:
#             batch_data = next(base_generator)
#             indices = base_generator.index_array
#             filenames = [base_generator.filenames[i] for i in indices]
#             yield batch_data[0], batch_data[1], filenames

# test_datagen = ImageDataGeneratorWithPaths(
#     rescale=1./255,  # Rescale the image pixel values to [0,1]
# )

# # Flow from dataframe method to load images using the dataframe
# test_original_generator = test_datagen.flow_from_dataframe(
#     dataframe=test_values, 
#     x_col='filepath',
#     target_size=(224, 224),   
#     color_mode='rgb',
#     class_mode= None, 
#     batch_size=32,
#     shuffle=False,
#     seed=42
# )

# # Get a batch of images, labels, and filenames
# images, labels, filenames = next(test_original_generator)

# # Now, filenames contains the filenames of the images in the current batch
# print(filenames)

# # Display the first two images
# plt.figure(figsize=(10, 5))

# plt.subplot(1, 2, 1)
# plt.imshow(images[0])
# plt.title('Image 1')
# #plt.gca().set_aspect('equal', adjustable='box')

# # plt.subplot(1, 2, 2)
# # plt.imshow(images[1])
# # plt.title('Image 2')
# # plt.gca().set_aspect('equal', adjustable='box')

# plt.show()

### Confirm that the generators have the correct image size of (height,width) = (224, 224)

In [ ]:
# import matplotlib.pyplot as plt

# # Get a batch of images and labels from the train generator
# images, labels = next(train_split_generator)

# # Print the shape, type of the images, and filepaths
# print("Images shape:", images.shape)
# print("Images type:", images.dtype)
# print("Filepaths:", train_split_generator.filenames[:2])  # Print the first two filepaths

# print(merged_train)

# # Display the first two images
# plt.figure(figsize=(10, 5))

# plt.subplot(1, 2, 1)
# plt.imshow(images[0])
# plt.title('Image 1')

# plt.subplot(1, 2, 2)
# plt.imshow(images[1])
# plt.title('Image 2')

# plt.show()

# # Do the same for the test generator
# images, labels = next(test_split_generator)
# print("Images shape:", images.shape)
# print("Images type:", images.dtype)
# print("Filepaths:", test_split_generator.filenames[:2])  # Print the first two filepaths

# # Display the first two images
# plt.figure(figsize=(10, 5))

# plt.subplot(1, 2, 1)
# plt.imshow(images[0])
# plt.title('Image 1')

# plt.subplot(1, 2, 2)
# plt.imshow(images[1])
# plt.title('Image 2')

# plt.show()

# **Model Development:**

## Model 1 of ResNet50. Score: 2.0596

In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output # This line gets the output of the base_model (ResNet50 without the top layer) and uses it as the starting point for adding new layers.
x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
predictions = Dense(8, activation='softmax')(x)  # Add a final output layer

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions) # this model links the ResNetModel and the new layers to the prediction layer

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'categorical_crossentropy'])

model.fit(train_split_generator, epochs=1, validation_data=test_split_generator)

predictions = model.predict(test_original_generator)

# Create a DataFrame with predictions
# Create a DataFrame with predictions
prediction_df = pd.DataFrame(predictions, columns=['antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent'])

# Add the 'id' from the original test DataFrame's index
prediction_df['id'] = test_values['id'] # id column from the original test set (test_values.csv)

# Ensure 'id' is the first column
prediction_df = prediction_df[['id'] + [col for col in prediction_df.columns if col != 'id']]

# Save the DataFrame to a CSV file
prediction_df.to_csv('submission_ResNet50_2.csv', index=False) # make sure to change file name to represent the model number
print("Successfully submitted!")

# Took 4 minutes for one epoch

2024-02-01 21:26:04.181091: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 21:26:04.406986: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 21:26:04.407048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 21:26:04.414191: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-01 21:26:04.414248: I external/local_xla/xla/stream_executor

140/140 [==============================] - 35s 250ms/step
Successfully submitted!


## Another thing to possibly try is to pick an even smaller 16:9 aspect ratio so that there is little to none upsampling (making images larger) 